# Deep Learning and Keras crash course

Daniil Merkulov

Skoltech

Prerequisites:
* `pip install numpy`
* `pip install tensorflow`
* `pip install pandas`

## Keras: краткий обзор
![](https://keras.io/img/keras-logo-small-wb.png)
* Библиотека для машинного обучения (прежде всего, обучения нейронных сетей, в т.ч. глубоких). 
* Представляет собой удобную обертку для мощных и хорошо оптимизированных вычислительных библиотек: TensorFlow, Theano
* Основные принципы: 
    1. Удобство использования
    2. Модульность
    3. Масштабируемость
    4. Работа с Python
    
Инструмент с низким порогом входа, подходящий как продвинутым исследовалетям, так и энтузиастам.

## Возвращение MNIST. Сравнения
Давайте вернемся к старой доброй задаче распознавания рукописных цифр, чтобы посмотреть сколько занимает код для такой нейросети, написанный на keras. Кроме того, здесь же мы попробуем собрать простую сверточную сеть для улучшения результата.

### Классическая нейросеть с плотными (dense) слоями.

In [1]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
import keras.datasets

Using TensorFlow backend.


In [2]:
# Загрузка данных. В keras уже есть несколько популярных датасетов, которые можно легко загрузить. Давайте загрузим MNIST
(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()

# Подготовка датасета: нормализация значений на [0,1] и перевод признаков в one-hot формат
X_train, X_test = X_train/255, X_test/255
y_train, y_test = keras.utils.to_categorical(y_train, 10), keras.utils.to_categorical(y_test, 10)
input_size = X_train[0].shape

In [3]:
# Создание модели. Sequential здесь означает последовательный тип модели, в который мы добавляем слои друг за другом
model = Sequential()

# Добавляем в стек модели слой за слоем. Полносвязный, активация и т.д.
# Важно: в первом слое Sequential модели keras необходимо указать размерность входных данных.
model.add(Flatten(input_shape=input_size))
model.add(Dense(units=256, input_shape=input_size))
model.add(Activation('relu'))
model.add(Dense(units=10))
model.add(Activation('softmax'))

In [4]:
# После описания архитектуры необходимо скомпилировать модель, указав минимизируемую функцию потерь, 
# оптимизатор и попросив модель выводить точность работы на тестовой выорке в процессе обучения
model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

# Тренировка с указанием данных, числа эпох и размера подвыборки
model.fit(X_train, y_train, epochs=5, batch_size=32)

Epoch 1/5
60000/60000 [==============================] - 8s - loss: 0.6187 - acc: 0.8470     
Epoch 2/5
60000/60000 [==============================] - 8s - loss: 0.3332 - acc: 0.9079     
Epoch 3/5
60000/60000 [==============================] - 8s - loss: 0.2851 - acc: 0.9205     
Epoch 4/5
60000/60000 [==============================] - 8s - loss: 0.2539 - acc: 0.9290     
Epoch 5/5
60000/60000 [==============================] - 8s - loss: 0.2297 - acc: 0.9355     


In [5]:
# Проверим качество работы модели на тестовых данных. Выводится loss и точночть.
model.evaluate(X_test, y_test)

 9952/10000 [============================>.] - ETA: 0s

[0.21461834994256496, 0.93879999999999997]

### Добавим сверточный слой и посмотрим на результат.

In [ ]:
from keras.layers import Conv2D
# Создание модели. Sequential здесь означает последовательный тип модели, в который мы добавляем слои друг за другом
conv_model = Sequential()

# Добавим явно число каналов в наш датасет - это важно для сверточных слоев. 
# т.е. делается преобразование (60000, 28, 28) -> (60000, 28, 28, 1). Это ничего не изменяет.
X_train, X_test = X_train.reshape((60000, 28, 28, 1)), X_test.reshape((10000, 28, 28, 1))
input_size = X_train[0].shape

# Здесь мы используем сверточный слой, который тренирует 32 фильтра размером 3x3 для поиска 
# конкретных геометрических (настраиваемых в процессе обучения)паттернов на входном изображении.
conv_model.add(Conv2D(24, (3, 3), input_shape=input_size))
conv_model.add(Activation('selu'))
conv_model.add(Flatten())
conv_model.add(Dense(64, activation='selu'))
conv_model.add(Dense(10, activation='softmax'))

In [ ]:
# Поставим другой оптимизатор для разнообразия
conv_model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Тренировка с указанием данных, числа эпох и размера подвыборки
conv_model.fit(X_train[:30000], y_train[:30000], epochs=3, batch_size=32)

Epoch 1/3
10816/30000 [=========>....................] - ETA: 34s - loss: 0.4272 - acc: 0.8696

In [ ]:
# Проверим качество работы модели на тестовых данных. Выводится loss и точночть.
conv_model.evaluate(X_test, y_test)

## Сверточные слои, dropout


Генерируются и обучаются несколько фильтров небольших размеров так, чтобы распознавать какие то характерные сочетания пикселей (паттерны). Ниже на картинке изображение 5x5 пикселей, фильтр имеет размеры 3x3. При этом на выходе такой операции имеем картинку такого же размера, в каждый из пикселей которого записан результат свертки (число) данного фильтра с картинкой при нахождении центра фильтра в этом пикселе. Для этого исходную картинку необходимо дополнить по краям. Обычно это делают либо нулями, либо дублируют ближайшие пиксели (padding)
![](convol.gif)

Чем глубже сверточный слой, тем более сложные паттерны он способен распознавать:
![](features.png)

Dropout - техника спасения нейросетей от переобучения, при которой в процессе тренировки случайно "выключаются" некоторые нейроны из моделей.

Альтернативный взгляд - вместо тренировки одной большой сети проходит одновременная тренировка нескольких подсетей меньшего размера, результаты которых потом усредняются (в каком то смысле, сглаживаются).
![](dropout.gif)

Давайте попробуем посмотреть, как написать сеть, состоящую из нескольких сверточных слоев на `keras`

In [ ]:
from keras.layers import MaxPooling2D, Dropout

# Все так же, создаем модель
cnn = Sequential()

# Начинаем со сверточного слоя, указывая тип активации на выходе из него и способ заполнения краев (padding)
cnn.add(Conv2D(64, (3, 3), input_shape=input_size, activation='selu', padding='same'))

# Здесь мы используем метод MaxPooling, который уменьшает размер обрабатываемого изображения, 
# выбирая из 4 пикселей 1 с максимальным значением, чтобы это быстрее считалось. (2,2) -> 1
cnn.add(MaxPooling2D(pool_size=(2, 2)))

# Слой dropout, который на каждом шаге "выключает" 25% случайно выбранных нейронов
cnn.add(Dropout(0.25))

# Еще сверточный слой
cnn.add(Conv2D(32, (3, 3), input_shape=input_size, activation='selu', padding='same'))
cnn.add(MaxPooling2D(pool_size=(2, 2)))

cnn.add(Dropout(0.5))

# Последний слой необходим для классификации, но перед ним необходимо векторизовать данные
cnn.add(Flatten())
cnn.add(Dense(10, activation='softmax'))


cnn.compile(loss='categorical_crossentropy',
                  optimizer = 'nadam',
                  metrics = ['accuracy'])

history_cnn = cnn.fit(X_train[:3000], y_train[:3000],
      batch_size=32,
      epochs=3,
      validation_data=(X_test, y_test))

## Дополнение данных в реальном времени
Большие модели требуют для обучения большого количества данных. Кроме того, иногда в задачах бывает крайне мало данных. В случае с изображениями в `keras` есть отличный инструмент для увеличения (раздувания) обучающей выборки.

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot

# Задаем параметры, в рамках которых выборка может дополняться. Эти параметры сильно зависят от выборки.
# Например, в задаче распознавания рукописных цифр отражение изображения не особо релевантно, т.к. в тестовой выборке таких встретиться не может
shift = 0.1 # максимальное значени сдвига в долях (от 0 до 1)
angle = 15   # максимальный угол поворота 

# Команда создает генератор, который при вызове в режиме реального времени генерирует необходимую подвыборку нужного размера
datagen = ImageDataGenerator(width_shift_range=shift, 
                             height_shift_range=shift, 
                             rotation_range=angle, 
                             horizontal_flip=False, 
                             vertical_flip=False,
                             featurewise_center=True)

# Подстраиваем генератор под наши данные
datagen.fit(X_train)

# Выберем случайно 9 картинок дополненной выборки и нарисуем их
for X_batch, y_batch in datagen.flow(X_train, y_train, batch_size=9):
    # создаем сетку 3х3
    for i in range(0, 9):
        pyplot.subplot(330 + 1 + i)
        pyplot.imshow(X_batch[i].reshape(28, 28), cmap=pyplot.get_cmap('gray'))
    # рисуем картинки
    pyplot.show()
    break

Вообще, функция `ImageDataGenerator` - очень мощный инструмент препроцессинга. Например, он позволяет стандартизировать (нормализовать на среднее 0 и стандартное отклонение на 1) изображения попиксельно (достаточно укзать `featurewise_std_normalization=True` как её аргумент), а так же уменьшать избыточность матрицы изображений (`zca_whitening=True`)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot

# Задаем параметры, в рамках которых выборка может дополняться. Эти параметры сильно зависят от выборки.
# Например, в задаче распознавания рукописных цифр отражение изображения не особо релевантно, т.к. в тестовой выборке таких встретиться не может
shift = 0.1 # максимальное значени сдвига в долях (от 0 до 1)
angle = 15   # максимальный угол поворота 

# Команда создает генератор, который при вызове в режиме реального времени генерирует необходимую подвыборку нужного размера
datagen = ImageDataGenerator(width_shift_range=shift, 
                             height_shift_range=shift, 
                             rotation_range=angle, 
                             horizontal_flip=False, 
                             vertical_flip=False,
                             featurewise_std_normalization=True,
                             zca_whitening=True)

# Подстраиваем генератор под наши данные
datagen.fit(X_train)

# Выберем случайно 9 картинок дополненной выборки и нарисуем их
for X_batch, y_batch in datagen.flow(X_train, y_train, batch_size=9):
    # создаем сетку 3х3
    for i in range(0, 9):
        pyplot.subplot(330 + 1 + i)
        pyplot.imshow(X_batch[i].reshape(28, 28), cmap=pyplot.get_cmap('gray'))
    # рисуем картинки
    pyplot.show()
    break

Иногда возникает необходимость сохранить "раздутую" выборку. Для этого можно воспользоваться следующими командами:

In [ ]:
import os

# Задаем параметры, в рамках которых выборка может дополняться. Эти параметры сильно зависят от выборки.
# Например, в задаче распознавания рукописных цифр отражение изображения не особо релевантно, т.к. в тестовой выборке таких встретиться не может
shift = 0.1 # максимальное значени сдвига в долях (от 0 до 1)
angle = 15   # максимальный угол поворота 

# Команда создает генератор, который при вызове в режиме реального времени генерирует необходимую подвыборку нужного размера
datagen = ImageDataGenerator(width_shift_range=shift, 
                             height_shift_range=shift, 
                             rotation_range=angle, 
                             horizontal_flip=False, 
                             vertical_flip=False)

# Подстраиваем генератор под наши данные
datagen.fit(X_train)

# Создаем папку, куда сохраним изображения
os.makedirs('images')

# Выберем случайно 9 картинок дополненной выборки и нарисуем их
for X_batch, y_batch in datagen.flow(X_train, y_train, batch_size=9, save_to_dir='images', save_prefix='aug', save_format='png'):
    # создаем сетку 3х3
    for i in range(0, 9):
        pyplot.subplot(330 + 1 + i)
        pyplot.imshow(X_batch[i].reshape(28, 28), cmap=pyplot.get_cmap('gray'))
    # рисуем картинки
    pyplot.show()
    break

Общие советы по дополнению выборки:
* Смотрите на исходную выборку
* Смотрите на раздутую выборку
* Пробуйте разные трансформации, иногда неожиданные сочетания могут дать заметный прирост

## Transfer Learning (Fine Tuning) или зачем изобретать велосипед, когда можно встать на плечи гигантов?

Идея: взять уже натренированную на большом датасете большую нейросеть и переделать её под себя.

In [ ]:
# Несмотря на то, что мы не будем тренировать бОльшую часть модели, код ниже будет работать долго на любом cpu
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D

# Стандартизированный размер картинок для загружаемой сети. Это важный параметр! 
img_width, img_height = 150, 150

train_data_dir = 'cats_dogs/train'
validation_data_dir = 'cats_dogs/validation'
nb_train_samples = 10000
nb_validation_samples = 1000
epochs = 3
batch_size = 16

# Загружаем модель VCG16
base_model = applications.VGG16(weights='imagenet', include_top=False)
print('Model loaded.')

# Создаем надстройку над ней для наших целей
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(64, activation='selu')(x)
predictions = Dense(2, activation='softmax')(x)

# Сшиваем модели в одну
model = Model(inputs=base_model.input, outputs=predictions)

# Замораживаем (не тренируем) первые слои сети, чтобы тренировать лишь оставшиеся быстрее
for layer in base_model.layers:
    layer.trainable = False

# Здесь неплохо указать небольшой (на порядок или два меньше стандартного) шаг алгоритма оптимизации для более аккуратного поиска.
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr=1e-2, momentum=0.9),
              metrics=['accuracy'])

# Раздуваем данные
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

# Дообучаем модель
model.fit_generator(
    train_generator,
    samples_per_epoch=nb_train_samples,
    epochs=epochs,
    validation_data=validation_generator,
    nb_val_samples=nb_validation_samples)

# ``` Важно, изображения должны быть организованы следующим образом:
# data/
#     train/
#         dogs/
#             dog001.jpg
#             dog002.jpg
#             ...
#         cats/
#             cat001.jpg
#             cat002.jpg
#             ...
#     validation/
#         dogs/
#             dog001.jpg
#             dog002.jpg
#             ...
#         cats/
#             cat001.jpg
#             cat002.jpg
#             ...
# ```



In [ ]:
model.summary()

## Домашнее задание

0. [2] Поиграться с кодом, доступном в семинаре, подергать за ручки, подобавлять слои, поизменять dropout rate, batch_size, тип активации, количество фильтров свертки и т.д. посмотреть результат
0. [2] Загрузите доступный в `keras` датасет `cifar10`, постройте сверточную нейросеть, которая дает 70 % точности на тренировочной выборке. 80%? 90%?
0. [4] Построить на `mnist` сверточную нейросеть, в которой будет расти размер сверточных фильтров, но уменьшаться их число. Натренировать такую сеть. Нарисовать полученные натренированные фильтры, посмотреть, какие паттерны они распознают в цифрах.
0. [5] Попробуйте с помощью `keras` и нейросетей решить задачу регрессии (или классификации), не связанную с обработкой изображений: например, загрузите датасет `boston_housing`, [доступный](https://keras.io/datasets/) в `keras`. 

## Полезные материалы
Хочется отметить, что на русском языке материалов пока несравненно меньше, чем на английском:
* [Официальная документация](https://keras.io/) - библиотека отлично документирована
* [Keras в конкретных примерах](https://github.com/tmheo/keras_exercise) - 25 отличных jupyter notebooks
* [Упражнения и примеры в Keras и TensorFlow](https://github.com/leriomaggio/deep-learning-keras-tensorflow)

In [ ]:
# Stylish cell, better to compile at the beginning
from IPython.core.display import HTML
def css_styling():
    styles = open("./styles/custom.css", "r").read()
    return HTML(styles)
css_styling()


# from IPython.html.services.config import ConfigManager
# from IPython.utils.path import locate_profile
# cm = ConfigManager(profile_dir=locate_profile(get_ipython().profile))
# cm.update('livereveal', {
#               'fontsize': 4,
#               'theme': 'simple_cyr',
#               'transition': 'zoom',
#               'start_slideshow_at': 'selected',
#               'height': '724',
#               'scroll': True,
#               'slideNumber': True
# })